In [1]:
import datetime


def singleton(class_):
    instances = {}

    def get_instance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return get_instance


@singleton
class Logger:
    def __init__(self, print_info=True, log_file=None):
        self.print_info = print_info
        self.log_file = open(log_file, 'r') if log_file is not None else None

    def __del__(self):
        if self.log_file:
            self.log_file.close()

    def log(self, obj=None, message=None):
        if message is None:
            return

        cur_time = str(datetime.datetime.now())
        obj_str = '[{}]'.format(obj) if obj is not None else ''
        msg_str = '[{}]{}{}'.format(cur_time, obj_str, message)
        if self.print_info:
            print(msg_str)
        if self.log_file:
            self.log_file.write(msg_str + '\n')

In [2]:
from collections import defaultdict
import pandas as pd
from enum import Enum


class Observable:
    def __init__(self):
        self.observer = None

    def register_observer(self, observer):
        self.observer = observer

    def notify_observer(self, event, *args, **kwargs):
        pass
        # self.observer.notify(event, *args, **kwargs)


class Events(Enum):
    EV_DAY_END = 1
    EV_INFECTION = 2
    EV_DEATH = 3
    EV_RECOVERY = 4
    EV_ANTIBODY = 5
    EV_HOSP_IN = 6
    EV_HOSP_OUT = 7
    EV_POLICY = 8


class Observer:
    def __init__(self, observables):
        self.observables = observables
        for obs in self.observables:
            obs.register_observer(self)

        self.infected_hist = []
        self.recovered_hist = []
        self.ab_hist = []
        self.dead_hist = []
        self.hospitalized_hist = []
        self.policies = []
        self.day = 0

        self.reset()

    def reset(self):
        self.infected = defaultdict(int)
        self.recovered = defaultdict(int)
        self.ab = defaultdict(int)
        self.dead = defaultdict(int)
        self.hositalized = 0

    def day_finished(self):
        self.infected_hist.append(self.infected)
        self.recovered_hist.append(self.recovered)
        self.ab_hist.append(self.ab)
        self.dead_hist.append(self.dead)
        self.hospitalized_hist.append(self.hositalized)
        self.day += 1
        self.reset()

    def infections_list(self):
        result = set()
        for lst in [self.infected_hist, self.recovered_hist, self.ab_hist, self.dead_hist]:
            for it in lst:
                result.update(list(it.keys()))
        return list(result)

    def export_df(self):
        infections_lst = self.infections_list()

        res = {
            'day': list(range(len(self.dead_hist))),
            'hospitalized': self.hospitalized_hist,
        }

        fmt = lambda x: x.name
        for lst in ['infected', 'recovered', 'ab', 'dead']:
            for infection_type in infections_lst:
                res[lst+'_'+fmt(infection_type)] = [d.get(infection_type, 0) for d in getattr(self, lst+'_hist')]

        res = pd.DataFrame(res)

        for lst in ['infected', 'recovered', 'ab', 'dead']:
            res[lst+'_all'] = sum([res[lst+'_'+fmt(inf_type)] for inf_type in infections_lst])

        return res

    def notify(self, event_type, *args, **kwargs):
        {
            Events.EV_DAY_END: self.notify_day_end,
            Events.EV_DEATH: self.notify_death,
            Events.EV_HOSP_IN: self.notify_hosp_in,
            Events.EV_HOSP_OUT: self.notify_host_out,
            Events.EV_RECOVERY: self.notify_recovery,
            Events.EV_ANTIBODY: self.notify_antibody,
            Events.EV_INFECTION: self.notify_infection,
            Events.EV_POLICY: self.notify_policy
        }[event_type](*args, **kwargs)

    def notify_policy(self, policy):
        self.policies.append((self.day, str(policy)))

    def notify_day_end(self, *args, **kwargs):
        Logger().log('Observer', '-' * 20 + ' Day end ' + '-' * 20)
        self.day_finished()

    def notify_infection(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New infection of type ' + str(infection_type.name))
        self.infected[infection_type] += 1

    def notify_death(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New death')
        self.dead[infection_type] += 1

    def notify_recovery(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New recovery of type ' + str(infection_type.name))
        self.recovered[infection_type] += 1

    def notify_antibody(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New antibody of type ' + str(infection_type.name))
        self.ab[infection_type] += 1

    def notify_hosp_in(self, *args, **kwargs):
        Logger().log('Observer', 'New hospitalization')
        self.hositalized += 1

    def notify_host_out(self, *args, **kwargs):
        Logger().log('Observer', 'Patient moved out from hospital')
        self.hositalized -= 1

In [3]:
from abc import ABC, abstractmethod
from random import expovariate
from enum import Enum

class Person:
    pass


class Infectable(ABC):
    def __init__(self, strength=1.0, contag=1.0):
        # contag is for contagiousness so we have less typos
        self.strength = strength
        self.contag = contag
        Logger().log('Infectable', 'New virus with stregth='+str(round(self.strength, 2)))

    @abstractmethod
    def cause_symptoms(self, person: Person):
        pass


class SeasonalFluVirus(Infectable):
    def cause_symptoms(self, person):
        person.temperature += 0.25

    @staticmethod
    def get_type():
        return InfectableType.SeasonalFlu


class SARSCoV2(Infectable):
    def cause_symptoms(self, person):
        person.temperature += 0.5

    @staticmethod
    def get_type():
        return InfectableType.SARSCoV2


class Cholera(Infectable):
    def cause_symptoms(self, person):
        person.water -= 1.0

    @staticmethod
    def get_type():
        return InfectableType.Cholera


class InfectableType(Enum):
    SeasonalFlu = 1
    SARSCoV2 = 2
    Cholera = 3


def get_infectable(infectable_type: InfectableType):
    if InfectableType.SeasonalFlu == infectable_type:
        return SeasonalFluVirus(strength=expovariate(10.0), contag=expovariate(10.0))

    elif InfectableType.SARSCoV2 == infectable_type:
        return SARSCoV2(strength=expovariate(0.42), contag=expovariate(0.42))

    elif InfectableType.Cholera == infectable_type:
        return Cholera(strength=expovariate(2.0), contag=expovariate(2.0))

    else:
        raise ValueError()


In [4]:
from abc import ABC, abstractmethod
from random import randint


class Drug(ABC):
    def apply(self, person):
        # somehow reduce person's symptoms
        pass


class AntipyreticDrug(Drug):
    pass


class Aspirin(AntipyreticDrug):
    """
        A cheaper version of the fever/pain killer.
    """

    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 0.5

    def apply(self, person):
        person.temperature = max(36.6, person.temperature - self.dose * self.efficiency)


class Ibuprofen(AntipyreticDrug):
    """A more efficient version of the fever/pain killer."""

    def __init__(self, dose):
        self.dose = dose

    def apply(self, person):
        person.temperature = 36.6


class RehydrationDrug(Drug):
    pass


class Glucose(RehydrationDrug):
    """A cheaper version of the rehydration drug."""

    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 0.1

    def apply(self, person):
        person.water = min(person.water + self.dose * self.efficiency,
                           0.6 * person.weight)


class Rehydron(RehydrationDrug):
    """A more efficient version of the rehydration drug."""

    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 1.0

    def apply(self, person):
        person._water = 0.6 * person.weight


class AntivirusDrug(Drug):
    pass


class Placebo(AntivirusDrug):
    def __init__(self, dose):
        self.dose = dose

    def apply(self, person):
        pass


class AntivirusSeasonalFlu(AntivirusDrug):
    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 1.0

    def apply(self, person):
        if isinstance(person.virus, SeasonalFluVirus):
            person.virus.strength -= self.dose * self.efficiency

        elif isinstance(person.virus, SARSCoV2):
            person.virus.strength -= self.dose * self.efficiency / 10.0


class AntivirusSARSCoV2(AntivirusDrug):
    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 0.1

    def apply(self, person):
        if isinstance(person.virus, SARSCoV2):
            person.virus.strength -= self.dose * self.efficiency


class AntivirusCholera(AntivirusDrug):
    def __init__(self, dose):
        self.dose = dose
        self.efficiency = 0.1

    def apply(self, person):
        if isinstance(person.virus, Cholera):
            person.virus.strength -= self.dose * self.efficiency


class DrugRepository(ABC):
    def __init__(self):
        self.treatment = []

    @abstractmethod
    def get_antifever(self, dose) -> Drug: pass

    @abstractmethod
    def get_rehydration(self, dose) -> Drug: pass

    @abstractmethod
    def get_seasonal_antivirus(self, dose) -> Drug: pass

    @abstractmethod
    def get_sars_antivirus(self, dose) -> Drug: pass

    @abstractmethod
    def get_cholera_antivirus(self, dose) -> Drug: pass

    def get_treatment(self):
        return self.treatment


class CheapDrugRepository(DrugRepository):
    def get_antifever(self, dose) -> Drug:
        return Aspirin(dose)

    def get_rehydration(self, dose) -> Drug:
        return Glucose(dose)

    def get_seasonal_antivirus(self, dose) -> Drug:
        return Placebo(dose)

    def get_sars_antivirus(self, dose) -> Drug:
        return Placebo(dose)

    def get_cholera_antivirus(self, dose) -> Drug:
        return Placebo(dose)


class ExpensiveDrugRepository(DrugRepository):
    def get_antifever(self, dose) -> Drug:
        return Ibuprofen(dose)

    def get_rehydration(self, dose) -> Drug:
        return Rehydron(dose)

    def get_seasonal_antivirus(self, dose) -> Drug:
        return AntivirusSeasonalFlu(dose)

    def get_sars_antivirus(self, dose) -> Drug:
        return AntivirusSARSCoV2(dose)

    def get_cholera_antivirus(self, dose) -> Drug:
        return AntivirusCholera(dose)

In [5]:
from abc import ABC, abstractmethod
from typing import List
from enum import Enum

class InfectableType(Enum):
    SeasonalFlu = 1
    SARSCoV2 = 2
    Cholera = 3


class AbstractPrescriptor(ABC):
    def __init__(self, drug_repository):
        self.drug_repository = drug_repository

    @abstractmethod
    def create_prescription(self) -> List[Drug]:
        pass


class SeasonalFluPrescriptor(AbstractPrescriptor):
    def __init__(self, drug_repository, antifever_dose, antivirus_dose):
        super().__init__(drug_repository)
        self.antifever_dose = antifever_dose
        self.antivirus_dose = antivirus_dose

    def create_prescription(self) -> List[Drug]:
        return [
            self.drug_repository.get_antifever(self.antifever_dose),
            self.drug_repository.get_seasonal_antivirus(self.antivirus_dose)
        ]


class CovidPrescriptor(AbstractPrescriptor):
    def __init__(self, drug_repository, antifever_dose, antivirus_dose):
        super().__init__(drug_repository)
        self.antifever_dose = antifever_dose
        self.antivirus_dose = antivirus_dose

    def create_prescription(self) -> List[Drug]:
        return [
            self.drug_repository.get_antifever(self.antifever_dose),
            self.drug_repository.get_sars_antivirus(self.antivirus_dose)
        ]


class CholeraPrescriptor(AbstractPrescriptor):
    def __init__(self, drug_repository, rehydradation_dose, antivirus_dose):
        super().__init__(drug_repository)
        self.rehydradation_dose = rehydradation_dose
        self.antivirus_dose = antivirus_dose

    def create_prescription(self) -> List[Drug]:
        return [
            self.drug_repository.get_rehydration(self.rehydradation_dose),
            self.drug_repository.get_cholera_antivirus(self.antivirus_dose)
        ]


def get_prescription_method(disease_type, drug_repository, dose1, dose2):
    if InfectableType.SeasonalFlu.value == disease_type.value:
        return SeasonalFluPrescriptor(drug_repository, dose1, dose2)
    elif InfectableType.SARSCoV2.value == disease_type.value:
        return CovidPrescriptor(drug_repository, dose1, dose2)
    elif InfectableType.Cholera.value == disease_type.value:
        return CholeraPrescriptor(drug_repository, dose1, dose2)
    else:
        raise ValueError()


In [6]:
import random

def singleton(class_):
    instances = {}

    def get_instance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return get_instance


class Policy:
    def __init__(self, strength=0.0):
        self.strength = strength
        assert (self.strength >= 0) and (self.strength <= 1)

    def __eq__(self, other):
        if self.__class__.__name__ == other.__class__.__name__:
            return self.strength == other.strength
        else:
            return False

    def __repr__(self):
        return '{}(p={:.2f})'.format(self.__class__.__name__, self.strength)

    def try_move(self, *args, **kwargs):
        return True

    def try_infect(self, *args, **kwargs):
        return True


class DistrictLockdownPolicy(Policy):
    def __init__(self, strength, max_dist=0.5):
        super(DistrictLockdownPolicy, self).__init__(strength)
        self.max_dist = max_dist
        assert (self.max_dist > 0) and (self.max_dist <= 1)

    def __eq__(self, other):
        if self.__class__.__name__ == other.__class__.__name__:
            return self.strength == other.strength and self.max_dist == other.max_dist
        else:
            return False

    def __repr__(self):
        return '{}(p={:.2f}, max_dist={:.2f})'.format(self.__class__.__name__, self.strength, self.max_dist)

    def try_move(self, old_pos, new_pos):
        min_j, max_j, min_i, max_i = GlobalContext().canvas
        dx = (abs(old_pos[0] - new_pos[0]) / (max_j-min_j)) ** 2
        dy = (abs(old_pos[1] - new_pos[1]) / (max_i-min_i)) ** 2
        dist = (dx+dy) ** 0.5  # Movement distance normalized by canvas size

        if random.random() > self.strength:
            # Some people ignore restrictions...
            return True
        else:
            # Other people move only inside their district
            return dist < self.max_dist


class TotalLockdownPolicy(Policy):
    def try_move(self, old_pos, new_pos):
        # Government issue total lockdown policy.
        # Only the most brave (desperate, stupid?) people move during day time
        # P(movement) = 1 - policy_strength
        return random.random() > self.strength


class PPEPolicy(Policy):
    def try_infect(self):
        # Government issue PPE usage policy (masks, gloves) which prevent infection.
        # But some people claims PPE are uncomfortable and don`t use it...?
        # P(infection) = 1 - policy_strength
        return random.random() > self.strength


class CombinedPolicy(Policy):
    def __init__(self, policies):
        super().__init__(1.0)
        self.policies = policies

    def __repr__(self):
        return ' + '.join([repr(p) for p in self.policies])

    def __eq__(self, other):
        if self.__class__.__name__ == other.__class__.__name__:
            if len(self.policies) != len(other.policies):
                return False
            for i in range(len(self.policies)):
                if self.policies[i] != other.policies[i]:
                    return False
            return True
        else:
            return False

    def try_move(self):
        # Assume that multiple policies gain total efficiency
        res = True
        for policy in self.policies:
            res = res & policy.try_move()
        return res

    def try_infect(self):
        # Assume that multiple policies gain total efficiency
        res = True
        for policy in self.policies:
            res = res & policy.try_infect()
        return res


@singleton
class DepartmentOfHealth(Observable):
    def __init__(self, hospitals):
        super().__init__()
        self.hospitals = hospitals

    def hospitalize(self, person):
        for hospital in self.hospitals:
            if len(hospital.patients) < hospital.capacity:
                hospital.patients.append(person)
                self.notify_observer(Events.EV_HOSP_IN)
                return hospital

        return None

    def make_policy(self):
        decision = GlobalContext().policy
        if len(GlobalContext().observer.infected_hist) > 0:
            # Collect statistics over all infections for the last day
            new_recoveries = sum(GlobalContext().observer.recovered_hist[-1].values())
            new_infections = sum(GlobalContext().observer.infected_hist[-1].values())
            new_death = sum(GlobalContext().observer.dead_hist[-1].values())
            population = len(GlobalContext().persons)
            # Make decisions
            if new_infections > 0.05 * population:
                decision = PPEPolicy(0.8)

            if new_recoveries > new_infections:
                decision = Policy(0.0)

            if new_death > 0.01 * population:
                decision = TotalLockdownPolicy(0.7)

        if decision != GlobalContext().policy:
            self.notify_observer(Events.EV_POLICY, decision)

        GlobalContext().policy = decision


@singleton
class GlobalContext:
    def __init__(self, canvas, persons, health_dept, observer=None):
        self.canvas = canvas
        self.persons = persons
        self.health_dept = health_dept
        self.policy = Policy(0.0)
        self.observer = observer


class Hospital:
    def __init__(self, capacity, drug_repository, doctor=None):
        self.doctor = doctor
        self.drug_repository = drug_repository
        self.capacity = capacity
        self.patients = []
        self.tests = []

    def __repr__(self):
        return 'Hospital(cap={}, n_patients={}, drug_repo={})'.format(self.capacity, len(self.patients),
                                                                      str(self.drug_repository.__class__.__name__))

    def _treat_patient(self, patient):
        if patient.virus is not None:
            disease_type = patient.virus.get_type()
            dose1, dose2 = random.random(), random.random()
            prescription_method = get_prescription_method(disease_type, self.drug_repository, dose1, dose2)
            prescription_drugs = prescription_method.create_prescription()

            fmt = lambda x: x.__class__.__name__
            Logger().log('Hospital', 'Treating patient infected by "{}" with [{}={}, {}={}]'.format(
                fmt(disease_type), fmt(prescription_drugs[0]), dose1, fmt(prescription_drugs[1]), dose2))

            for drug in prescription_drugs:
                drug.apply(patient)

    def release_patient(self, person):
        try:
            idx = self.patients.index(person)
            del self.patients[idx]
        except IndexError:
            pass

    def treat_patients(self):
        Logger().log('Hospital', 'Treating all patients...')
        for patient in self.patients:
            self._treat_patient(patient)

In [7]:
from abc import ABC, abstractmethod

class Person:
    pass


def dist(pos1, pos2):
    min_j, max_j, min_i, max_i = GlobalContext().canvas
    dx = ((pos1[0]-pos2[0]) / (max_j-min_j)) ** 2
    dy = ((pos1[1]-pos2[1]) / (max_i-min_i)) ** 2
    return (dx + dy) ** 0.5


class State(ABC):
    def __init__(self, person):
        self.person = person

    @abstractmethod
    def day_actions(self): pass

    @abstractmethod
    def night_actions(self): pass

    @abstractmethod
    def interact(self, other): pass

    @abstractmethod
    def get_infected(self, virus): pass


class Healthy(State):
    def day_actions(self):
        self.person._day_actions()

    def night_actions(self):
        self.person.position = self.person.home_position

    def interact(self, other: Person): pass

    def get_infected(self, virus):
        if virus.get_type() not in self.person.antibody_types:
            Logger().log('Healthy', self.person.antibody_types)
            self.person.virus = get_infectable(virus.get_type())
            self.person.set_state(AsymptomaticSick(self.person))


class AsymptomaticSick(State):
    DAYS_SICK_TO_FEEL_BAD = 4

    def __init__(self, person):
        super().__init__(person)
        self.days_sick = 0

    def day_actions(self):
        self.person._day_actions()

        if self.person.is_life_incompatible_condition():
            self.person.set_state(Dead(self.person))

    def night_actions(self):
        self.person.position = self.person.home_position
        if self.days_sick == AsymptomaticSick.DAYS_SICK_TO_FEEL_BAD:
            self.person.set_state(SymptomaticSick(self.person))
        self.days_sick += 1

    def interact(self, other):
        if GlobalContext().policy.try_infect():
            other.get_infected(self.person.virus)

    def get_infected(self, virus): pass


class SymptomaticSick(State):
    def __init__(self, person):
        super().__init__(person)
        self.person.notify_observer(Events.EV_INFECTION, person.virus.get_type())

    def day_actions(self):
        self.person.progress_disease()

        if self.person.is_life_threatening_condition() and (self.person.hospital is None):
            health_dept = DepartmentOfHealth(None)
            self.person.hospital = health_dept.hospitalize(self.person)

        if self.person.is_life_incompatible_condition():
            self.person.set_state(Dead(self.person))

    def night_actions(self):
        # try to fight the virus
        if self.person.virus.strength <= 0:
            self.person.set_state(Healthy(self.person))
            self.person.antibody_types.add(self.person.virus.get_type())

            self.person.notify_observer(Events.EV_RECOVERY, self.person.virus.get_type())
            self.person.notify_observer(Events.EV_ANTIBODY, self.person.virus.get_type())
            self.person.virus = None

            if self.person.hospital:
                self.person.hospital.release_patient(self.person)
                self.person.hospital = None
                self.person.notify_observer(Events.EV_HOSP_OUT)

    def interact(self, other):
        pass

    def get_infected(self, virus):
        pass


class Dead(State):
    def __init__(self, person):
        super().__init__(person)
        self.person.notify_observer(Events.EV_DEATH, self.person.virus.get_type())

        if self.person.hospital:
            self.person.hospital.release_patient(self.person)
            self.person.hospital = None
            self.person.notify_observer(Events.EV_HOSP_OUT)

    def day_actions(self): pass

    def night_actions(self): pass

    def interact(self, other): pass

    def get_infected(self, virus): pass


class Person(Observable):
    MAX_TEMPERATURE_TO_SURVIVE = 44.0
    LOWEST_WATER_PCT_TO_SURVIVE = 0.4

    LIFE_THREATENING_TEMPERATURE = 40.0
    LIFE_THREATENING_WATER_PCT = 0.5

    def __init__(self, home_position=(0, 0), age=30, weight=70):
        super().__init__()
        self.virus = None
        self.antibody_types = set()
        self.temperature = 36.6
        self.weight = weight
        self.water = 0.6 * self.weight
        self.age = age
        self.home_position = home_position
        self.position = home_position
        self.state = Healthy(self)
        self.hospital = None

    def attrs(self):
        return {
            'virus': self.virus,
            'antibodies': [ab.name for ab in self.antibody_types],
            'temp': round(self.temperature, 1),
            'weight': round(self.weight, 1),
            'water': round(self.water, 1),
            'age': self.age,
            'home': self.home_position,
            'pos': self.position,
            'state': self.state.__class__.__name__
        }

    def __repr__(self):
        return '{}({})'.format(self.__class__.__name__,
                               ', '.join(['{}={}'.format(k, v) for k, v in self.attrs().items()]))

    def day_actions(self):
        self.state.day_actions()

    def _day_actions(self):
        pass

    def night_actions(self):
        self.state.night_actions()

    def interact(self, other):
        self.state.interact(other)

    def get_infected(self, virus):
        self.state.get_infected(virus)

    def is_close_to(self, other):
        return dist(self.position, other.position) <= 0.01

    def fightvirus(self):
        if self.virus:
            self.virus.strength -= (3.0 / self.age)
            # Logger().log('Person', 'New virus strength = '+str(self.virus.strength))

    def progress_disease(self):
        if self.virus:
            self.virus.cause_symptoms(self)

    def set_state(self, state):
        self.state = state

    def is_life_threatening_condition(self):
        return self.temperature >= Person.LIFE_THREATENING_TEMPERATURE or \
               self.water / self.weight <= Person.LIFE_THREATENING_WATER_PCT

    def is_life_incompatible_condition(self):
        return self.temperature >= Person.MAX_TEMPERATURE_TO_SURVIVE or \
               self.water / self.weight <= Person.LOWEST_WATER_PCT_TO_SURVIVE

In [8]:
from random import randint
from abc import ABC, abstractmethod

class DefaultPerson(Person):
    def _day_actions(self):
        min_j, max_j, min_i, max_i = GlobalContext().canvas
        new_position = (randint(min_j, max_j), randint(min_i, max_i))

        if GlobalContext().policy.try_move(self.position, new_position):
            self.position = new_position


class CommunityPerson(Person):
    def __init__(self, community_position=(0, 0), **kwargs):
        super().__init__(**kwargs)
        self.community_position = community_position

    def attrs(self):
        basic_attrs = super(CommunityPerson, self).attrs()
        basic_attrs['community'] = self.community_position
        return basic_attrs

    def _day_actions(self):
        if GlobalContext().policy.try_move(self.position, self.community_position):
            self.position = self.community_position


class AbstractPersonFactory(ABC):
    def __init__(self, context):
        self.min_age, self.max_age = 1, 90
        self.min_weight, self.max_weight = 30, 120
        self.min_j, self.max_j, self.min_i, self.max_i = context

    @abstractmethod
    def get_person(self) -> Person:
        pass


class DefaultPersonFactory(AbstractPersonFactory):
    def get_person(self) -> Person:
        return DefaultPerson(
            home_position=(randint(self.min_j, self.max_j), randint(self.min_i, self.max_i)),
            age=randint(self.min_age, self.max_age),
            weight=randint(self.min_weight, self.max_weight),
        )


class CommunityPersonFactory(AbstractPersonFactory):
    def __init__(self, *args, community_position=(0, 0)):
        super().__init__(*args)
        self.community_position = community_position

    def get_person(self) -> Person:
        return CommunityPerson(
            home_position=(randint(self.min_j, self.max_j), randint(self.min_i, self.max_i)),
            age=randint(self.min_age, self.max_age),
            weight=randint(self.min_weight, self.max_weight),
            community_position=self.community_position
        )

In [9]:
from collections import defaultdict
import pandas as pd
from enum import Enum

class Observable:
    def __init__(self):
        self.observer = None

    def register_observer(self, observer):
        self.observer = observer

    def notify_observer(self, event, *args, **kwargs):
        pass
        # self.observer.notify(event, *args, **kwargs)


class Events(Enum):
    EV_DAY_END = 1
    EV_INFECTION = 2
    EV_DEATH = 3
    EV_RECOVERY = 4
    EV_ANTIBODY = 5
    EV_HOSP_IN = 6
    EV_HOSP_OUT = 7
    EV_POLICY = 8


class Observer:
    def __init__(self, observables):
        self.observables = observables
        for obs in self.observables:
            obs.register_observer(self)

        self.infected_hist = []
        self.recovered_hist = []
        self.ab_hist = []
        self.dead_hist = []
        self.hospitalized_hist = []
        self.policies = []
        self.day = 0

        self.reset()

    def reset(self):
        self.infected = defaultdict(int)
        self.recovered = defaultdict(int)
        self.ab = defaultdict(int)
        self.dead = defaultdict(int)
        self.hositalized = 0

    def day_finished(self):
        self.infected_hist.append(self.infected)
        self.recovered_hist.append(self.recovered)
        self.ab_hist.append(self.ab)
        self.dead_hist.append(self.dead)
        self.hospitalized_hist.append(self.hositalized)
        self.day += 1
        self.reset()

    def infections_list(self):
        result = set()
        for lst in [self.infected_hist, self.recovered_hist, self.ab_hist, self.dead_hist]:
            for it in lst:
                result.update(list(it.keys()))
        return list(result)

    def export_df(self):
        infections_lst = self.infections_list()

        res = {
            'day': list(range(len(self.dead_hist))),
            'hospitalized': self.hospitalized_hist,
        }

        fmt = lambda x: x.name
        for lst in ['infected', 'recovered', 'ab', 'dead']:
            for infection_type in infections_lst:
                res[lst+'_'+fmt(infection_type)] = [d.get(infection_type, 0) for d in getattr(self, lst+'_hist')]

        res = pd.DataFrame(res)

        for lst in ['infected', 'recovered', 'ab', 'dead']:
            res[lst+'_all'] = sum([res[lst+'_'+fmt(inf_type)] for inf_type in infections_lst])

        return res

    def notify(self, event_type, *args, **kwargs):
        {
            Events.EV_DAY_END: self.notify_day_end,
            Events.EV_DEATH: self.notify_death,
            Events.EV_HOSP_IN: self.notify_hosp_in,
            Events.EV_HOSP_OUT: self.notify_host_out,
            Events.EV_RECOVERY: self.notify_recovery,
            Events.EV_ANTIBODY: self.notify_antibody,
            Events.EV_INFECTION: self.notify_infection,
            Events.EV_POLICY: self.notify_policy
        }[event_type](*args, **kwargs)

    def notify_policy(self, policy):
        self.policies.append((self.day, str(policy)))

    def notify_day_end(self, *args, **kwargs):
        Logger().log('Observer', '-' * 20 + ' Day end ' + '-' * 20)
        self.day_finished()

    def notify_infection(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New infection of type ' + str(infection_type.name))
        self.infected[infection_type] += 1

    def notify_death(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New death')
        self.dead[infection_type] += 1

    def notify_recovery(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New recovery of type ' + str(infection_type.name))
        self.recovered[infection_type] += 1

    def notify_antibody(self, infection_type, *args, **kwargs):
        Logger().log('Observer', 'New antibody of type ' + str(infection_type.name))
        self.ab[infection_type] += 1

    def notify_hosp_in(self, *args, **kwargs):
        Logger().log('Observer', 'New hospitalization')
        self.hositalized += 1

    def notify_host_out(self, *args, **kwargs):
        Logger().log('Observer', 'Patient moved out from hospital')
        self.hositalized -= 1

# Tests

In [34]:
import unittest

class Test3(unittest.TestCase):
    def setUp(self):
        self.antibodied_person = Person()
        self.antibodied_person.antibody_types.add(InfectableType.SARSCoV2)
        self.infected_person = Person()
        self.infected_person.get_infected(get_infectable(InfectableType.SARSCoV2))
        self.antibodied_person.interact(self.infected_person)

    def test_ok(self):
        self.assertTrue(self.antibodied_person.state == Healthy)
        
class Test5(unittest.TestCase): 
    def setUp(self):
        self.infected_person = Person()
        self.infected_person.get_infected(get_infectable(InfectableType.SARSCoV2))
        self.infected_person.set_state(SymptomaticSick(self.infected_person))
        self.infected_person.virus.strength = 0
        self.infected_person.night_actions()
    
    def test_ok(self):
        self.assertTrue(isinstance(self.infected_person.state, Healthy))
        
class Test4(unittest.TestCase): 
    def setUp(self):
        self.infected_person = Person()
        self.infected_person.get_infected(random.choice([SeasonalFluVirus,SARSCoV2,Cholera]))
        self.infected_person.set_state(AsymptomaticSick(self.infected_person))
        self.infected_person.virus.strength = 1
        self.infected_person.days_sick = 3
    
    def test_ok(self):
        self.assertTrue(isinstance(self.infected_person.state, SymptomaticSick))
        
class Test6(unittest.TestCase): 
    def setUp(self):
        self.infected_person = Person()
        self.infected_person.get_infected(random.choice([SeasonalFluVirus,SARSCoV2,Cholera]))     
        self.infected_person.set_state(SymptomaticSick(self.infected_person))
        self.infected_person.virus.strength = 1
        self.infected_person.days_sick = 4
        self.infected_person.water = 0.3
        self.infected_person.temperature = 45
    
    def test_ok(self):
        self.assertTrue(isinstance(self.infected_person.state, Dead))
        